This is the code that I will be adding to! Please feel free to fork/edit this : )

### Importing modules

In [1]:
import os
import glob
import numpy as np
import pandas as pd
import librosa
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import wave
from keras.preprocessing import image
from PIL import Image
import cv2 

%matplotlib inline


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)
tf.config.list_physical_devices('GPU')


[]

### Setting file locations

In [3]:
DATA_DIR = 'C:/Users/Farrell Joswara/Documents/GitHub/AI/data'  # Replace with your dataset path
REAL_DIR = os.path.join(DATA_DIR, 'real')
FAKE_DIR = os.path.join(DATA_DIR, 'fake')


### Defining Functions

In [5]:
def create_spectrogram(audio_file, image_file): 
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1)

    y, sr = librosa.load(audio_file)
    if y.size == 0:
        raise ValueError
    ms = librosa.feature.melspectrogram(y=y, sr=sr)
    log_ms = librosa.power_to_db(ms, ref=np.max)
    

    log_ms = (log_ms-np.min(log_ms))/(np.max(log_ms)-np.min(log_ms)) #normalizes by taking the lowest value and then sets that to zero, then divides the whole thing by the highest value to get a scale from 0 to 1
    librosa.display.specshow(log_ms, sr=sr, cmap='gray')

    fig.savefig(image_file)
    plt.close(fig)
    
def create_pngs_from_wavs(input_path, output_path): #ensure that you set the input path to the folder where the wav files are, and then the output directory will create a folder if one isn't already there and input the images there
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    dir = os.listdir(input_path)

    for i, file in enumerate(dir):
        input_file = os.path.join(input_path, file)
        output_file = os.path.join(output_path, file.replace('.wav', '.png'))
        create_spectrogram(input_file, output_file)

In [ ]:
create_pngs_from_wavs(REAL_DIR, 'Spectrograms/real')
create_pngs_from_wavs(FAKE_DIR, 'Spectrograms/fake')


# Prearing the dataset

In [6]:
path = r"C:\Users\Farrell Joswara\Documents\GitHub\MVI-Deepfake"
dir = os.path.join(path,"Spectrograms")
batch_size = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
  dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  batch_size=batch_size)

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


Found 44316 files belonging to 2 classes.
Using 35453 files for training.
Found 44316 files belonging to 2 classes.
Using 8863 files for validation.


In [5]:
import tensorflow as tf

# Check if a GPU is available
if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is not available")


GPU is not available


In [28]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [7]:
import tensorflow as tf

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(256, 256, 3)),  # Input shape must match your data
    tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same'),  # Increased filters
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same'),  # Increased filters
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),  # Increased filters
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same'),  # Increased filters
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),  # Increased units
    tf.keras.layers.Dropout(0.5),  # Dropout for regularization
    tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train the model with Early Stopping
history = model.fit(
    train_ds,         # Ensure this is a properly defined TensorFlow Dataset
    validation_data=val_ds,  # Ensure this is a properly defined TensorFlow Dataset
    epochs=20,        # Increased number of epochs
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    ]
)

# Optionally: Evaluate the model
# test_loss, test_accuracy = model.evaluate(test_ds)
# print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")


c:\Users\Farrell Joswara\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\preprocessing\tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
1108/1108 ━━━━━━━━━━━━━━━━━━━━ 6571s 6s/step - accuracy: 0.7934 - loss: 0.7168 - val_accuracy: 0.9523 - val_loss: 0.1274
Epoch 2/20
 396/1108 ━━━━━━━━━━━━━━━━━━━━ 54:37 5s/step - accuracy: 0.9334 - loss: 0.1599

AbortedError: Graph execution error:

Detected at node StatefulPartitionedCall/sequential_1/conv2d_3_1/Relu defined at (most recent call last):
<stack traces unavailable>
Operation received an exception:Status: 1, message: could not create a memory object, in file tensorflow/core/kernels/mkl/mkl_conv_ops.cc:1112
	 [[{{node StatefulPartitionedCall/sequential_1/conv2d_3_1/Relu}}]] [Op:__inference_one_step_on_iterator_4343]

In [26]:
import cv2
import numpy as np
import tensorflow as tf

# Load and resize images
real = cv2.imread(r'C:\Users\Farrell Joswara\Documents\GitHub\MVI-Deepfake\Spectrograms\real\DF_E_2085402.png')
fake = cv2.imread(r'C:\Users\Farrell Joswara\Documents\GitHub\MVI-Deepfake\Spectrograms\fake\DF_E_2000026.png')

# Resize the images to (256, 256) as expected by the model
resize_size = (256, 256)
real_resized = cv2.resize(real, resize_size)
fake_resized = cv2.resize(fake, resize_size)

# Normalize the images
real_normalized = real_resized / 255.0
fake_normalized = fake_resized / 255.0

# Expand dimensions to match the batch size
real_input = np.expand_dims(real_normalized, axis=0)  # Shape (1, 256, 256, 3)
fake_input = np.expand_dims(fake_normalized, axis=0)  # Shape (1, 256, 256, 3)

# Make predictions
real_yhat = model.predict(real_input)
fake_yhat = model.predict(fake_input)

print(f"Real: {real_yhat}")
print(f"Fake: {fake_yhat}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Real: [[0.01556102]]
Fake: [[0.01509224]]


In [8]:
real_files = []
fake_files = []
for files in os.listdir('Spectrograms/real'):
    real_files.append(os.path.join('Spectrograms/real',files))
for files in os.listdir('Spectrograms/fake'):
    fake_files.append(os.path.join('Spectrograms/fake',files))


minimum_samples = min(len(real_files),len(fake_files))
np.random.seed(1)


temp = np.concatenate((np.array(real_files),np.random.choice(fake_files,minimum_samples,replace=False)))

all_files = []
for file in temp:
    print(file)
    image = (cv2.imread(file))
    all_files.append(cv2.resize(image,(128, 128)))
#print(len(all_files))






Spectrograms/real\DF_E_2000053.png
Spectrograms/real\DF_E_2000058.png
Spectrograms/real\DF_E_2000079.png
Spectrograms/real\DF_E_2000246.png
Spectrograms/real\DF_E_2000503.png
Spectrograms/real\DF_E_2000508.png
Spectrograms/real\DF_E_2000531.png
Spectrograms/real\DF_E_2000676.png
Spectrograms/real\DF_E_2001149.png
Spectrograms/real\DF_E_2001163.png
Spectrograms/real\DF_E_2001230.png
Spectrograms/real\DF_E_2001695.png
Spectrograms/real\DF_E_2001832.png
Spectrograms/real\DF_E_2002152.png
Spectrograms/real\DF_E_2002202.png
Spectrograms/real\DF_E_2002307.png
Spectrograms/real\DF_E_2002420.png
Spectrograms/real\DF_E_2002526.png
Spectrograms/real\DF_E_2002672.png
Spectrograms/real\DF_E_2002753.png
Spectrograms/real\DF_E_2002772.png
Spectrograms/real\DF_E_2002884.png
Spectrograms/real\DF_E_2002979.png
Spectrograms/real\DF_E_2003008.png
Spectrograms/real\DF_E_2003021.png
Spectrograms/real\DF_E_2003082.png
Spectrograms/real\DF_E_2003108.png
Spectrograms/real\DF_E_2003222.png
Spectrograms/real\DF

In [21]:
values = ([1]*len(real_files))
values = values + ([0]*len(real_files))

print(len(values),len(all_files))

data = {
    "files":all_files,
    "value":values
}
df = pd.DataFrame(data)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)


X = df[df.columns[:-1]].values
Y = df[df.columns[-1]].values
Y = np.asarray(Y).astype(np.float32)

'''

1 = real
0 = fake 

'''

43462 43462
[array([[[ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0],
         ...,
         [ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0]],

        [[ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0],
         ...,
         [ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0]],

        [[ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0],
         ...,
         [ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0]],

        ...,

        [[18, 18, 18],
         [18, 18, 18],
         [48, 48, 48],
         ...,
         [ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0]],

        [[ 0,  0,  0],
         [ 0,  0,  0],
         [11, 11, 11],
         ...,
         [ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0]],

        [[ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0],
         ...,
         [ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0]]], dtype=uint8)]


'\n\n1 = real\n0 = fake \n\n'

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 4)


In [17]:
batch_size = 16
nb_classes =4
nb_epochs = 5
img_rows, img_columns = 128, 128
img_channel = 3
nb_filters = 32
nb_pool = 2
nb_conv = 3

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu,
                           input_shape=(128, 128, 1)),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation=tf.nn.relu),
    tf.keras.layers.Dense(1,  activation=tf.nn.sigmoid)
])
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(X_train, y_train, batch_size = batch_size, epochs = nb_epochs, verbose = 1, validation_data = (X_test, y_test))
score = model.evaluate(X_test, y_test, verbose = 0 )
print("Test Score: ", score[0])
print("Test accuracy: ", score[1])

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).